# import packages and codes

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# code import, make sure the path is correct, or .py is located in the same folder
import DBPS_BLR_util as d_util
import SBPS_BLR_util as s_util
import BPS_BLR_util as b_util
import MH_BLR_util as mh_util

import seaborn as sns
import os
import gc
import numbers
import time
import math
from statsmodels.graphics.tsaplots import plot_acf

# for log
import logging
from imp import reload
reload(logging)


<module 'logging' from '/home/user/anaconda3/lib/python3.7/logging/__init__.py'>

# Shared hyper-parameters

In [2]:
iterations = 100000
burninIters = iterations / 10 * 3
data_size = [16384, 32768, 65536, 131072, 262144, 524288]
verbose = 1e05
MH_settings = []
BPS_settings = []
SBPS_settings = []
DBPS_settings = []

+ MH_settings:
 1. execution time AFTER burn-in
 2. total execution time
 3. sample counts AFTER burn-in ends
 4. acceptance rate
+ BPS_settings:
 1. execution time AFTER burn-in
 2. total execution time
 3. storage time cost AFTER burn-in
 4. total storage time cost
 5. sample counts AFTER burn-in
 6. acceptance rate of prior bounce (prior bounce event / total iteration)
 7. acceptance rate of likelihood bound (likelihood bounce event / total iteration)
 8. refresh rate (refresh event / total iteration)
+ SBPS_settings:
 1. execution time AFTER burn-in
 2. total execution time
 3. storage time cost AFTER burn-in
 4. total storage time cost
 5. sample counts AFTER burn-in
 6. acceptance rate of bounce event (bounce event / total iteration)
+ DBPS_settings:
 1. execution time AFTER burn-in
 2. total execution time
 3. sample counts AFTER burn-in
 4. acceptance rate of stage1 (stage1 accepted event / total iteration)
 5. acceptance rate of stage2 (stage2 accepted event / total iteration)
 
Every algorithm will generate the sample with filename "(algorithm name)_BLR_iterations_(iterations)_datasize(datasize).npy"   <br/>
Change the filename in each .py file if you need

In [11]:
for size in data_size:
    logging.basicConfig(level=logging.NOTSET,filemode='a',format='%(asctime)s - %(levelname)s : %(message)s',  filename='log.txt')
    # comment out these np.load if you want to reset those settings
#     MH_settings = np.load('MHsetting.npy').tolist()
#     BPS_settings = np.load('BPSsetting.npy').tolist()
#     SBPS_settings = np.load('SBPSsetting.npy').tolist()
#     DBPS_settings = np.load('DBPSsetting.npy').tolist()

    # comment out these codes if you want to use exist data
#     N = size
#     X = np.random.normal(0, 1, N)
#     true_beta = [0.2,0.8]
#     true_p = b_util.expit(true_beta[0] + true_beta[1] * X)
#     Y = np.random.binomial(1, true_p)
#     np.save('X_' + str(N), X)
#     np.save('Y_' + str(N), Y)
    
    X = np.load('X_'+str(size)+'.npy')
    Y = np.load('Y_'+str(size)+'.npy')
    
    # metropolis-hastings
    logging.debug('MH start, size : ' + str(size))
    b_prior_sd = 1
    can_sd = 0.5
    store_skip = 1
    a = time.time()
    BLR_MH = mh_util.MH(X, Y, b_prior_sd)
    BLR_MH.sampler(can_sd, burninIters, iterations, store_skip, verbose)
    b = time.time()
    
    np.save('mh_BLR_iterations_2e06_datasize_' + str(size), BLR_MH.all_samples)
    runtime = b-BLR_MH.burnin_time
    totaltime = b-a
    MH_settings.append([runtime, totaltime, BLR_MH.burnin_sample, BLR_MH.p])
    del BLR_MH
    gc.collect()
    logging.debug('MH end, size : ' + str(size))
    
    # DBPS
    logging.debug('DBPS start, size : ' + str(size))
    delta = 10**-1.5
    store_skip = 1
    kappa = 10**-2.5
    subset = 0
    sto = 0
    a = time.time()
    BLR_DBPS = d_util.DBPS_BLR(X, Y, delta, store_skip)
    BLR_DBPS.DBPS_sampler(burninIters, iterations, verbose, kappa, subset, sto)
    b = time.time()
    
    np.save('dbps_BLR_iterations_2e06_datasize_' + str(size), BLR_DBPS.all_beta)
    runtime = b-BLR_DBPS.burnin_time
    totaltime = b-a
    DBPS_settings.append([runtime, totaltime, BLR_DBPS.burnin_sample, BLR_DBPS.stage1_accept, BLR_DBPS.stage2_accept])
    del BLR_DBPS
    gc.collect()
    logging.debug('DBPS end, size : ' + str(size))
    
    # SBPS
    logging.debug('SBPS start, size : ' + str(size))
    up_factor = 3
    batch_size = math.ceil(size/100)
    T = 10000000
    dt = 0.5
    prior_mean = 0
    priorG_var = 1e10
    sbps_sample_interval = 2e-04
    a = time.time()
    BLR_SBPS = s_util.SBPS(X,Y,T,dt,up_factor,prior_mean,priorG_var,batch_size, verbose, iterations, burninIters)
    BLR_SBPS.generate_samples(sbps_sample_interval, use_preconditioner = False)
    b = time.time()
    
    np.save('sbps_BLR_iterations_2e06_datasize_' + str(size), BLR_SBPS.alt_samples)
    runtime = b-BLR_SBPS.burnin_time
    totaltime = b-a
    SBPS_settings.append([runtime, totaltime, BLR_SBPS.after_burnin_storage_time, BLR_SBPS.all_storage_time, BLR_SBPS.burnin_sample, BLR_SBPS.bounce_counts])
    del BLR_SBPS
    gc.collect()
    logging.debug('SBPS end, size : ' + str(size))
    
    # BPS
    logging.debug('BPS start, size : ' + str(size))
    delta = 0.5
    ref = 0.5
    prior_var = 1
    store_skip = 1
    bps_sample_time = 2e-04
    a = time.time()
    BLR_BPS = b_util.BPS(X, Y, prior_var, ref, store_skip)
    BLR_BPS.sampler(delta, bps_sample_time, iterations, burninIters, verbose)
    b = time.time()
    
    np.save('bps_BLR_iterations_2e06_datasize_' + str(size), BLR_BPS.all_samples)
    runtime = b-BLR_BPS.burnin_time
    totaltime = b-a
    BPS_settings.append([runtime, totaltime, BLR_BPS.after_burnin_storage_time, BLR_BPS.all_storage_time, BLR_BPS.burnin_sample, BLR_BPS.prior_bounce, BLR_BPS.post_bounce, BLR_BPS.ref_count])
    del BLR_BPS
    gc.collect()
    logging.debug('BPS end, size : ' + str(size))
    
    np.save('MHsetting', np.array(MH_settings))
    np.save('DBPSsetting', np.array(DBPS_settings))
    np.save('SBPSsetting', np.array(SBPS_settings))
    np.save('BPSsetting', np.array(BPS_settings))